In [2]:
import docker
client = docker.from_env()

In [ ]:
container = client.containers.run("alpine", "sleep 100", remove=True, detach=True)
result = container.exec_run("cat", stdin=True, socket=True)
socket = result.output

In [23]:
socket._sock.sendall(b"Hello World\n")

In [25]:
socket._sock.recv(1024)

b''

In [29]:
socket.close()

In [33]:
container.id

'47e6d89bd0ddfcbaa524bbd83142cc2a19e99af871e91f97eeddbfd211c0f41f'

In [ ]:
import socket
code = """\
#include "nanobench.h"
#include <iostream>
#include <fstream>

struct Point { int x, y; };

void func() {
    const int N = 1000;
    std::vector<std::vector<int>> matrix(N, std::vector<int>(N, 1));
    long long sum = 0;
    // Column-major traversal
    for (int j = 0; j < N; ++j) {
        for (int i = 0; i < N; ++i) {
            sum += matrix[i][j];
        }
    }
    std::cout << sum << std::endl;
}

int main() {
    std::ofstream nullStream("/dev/null");
    auto* oldBuf = std::cout.rdbuf(nullStream.rdbuf());
    
    ankerl::nanobench::Bench()
    .output(&std::cerr)
    .run("test", []() {
        func();
    });

    std::cout.rdbuf(oldBuf);
}
"""

compileCommand = (
    "g++ "
    "-O3 --std=c++20 "
    "nanobench.o "
    "-x c++ - "
    "-o a.out"
)

container = client.containers.run("localhost/benchmark-examples", "sleep 100", remove=True, detach=True)
res = container.exec_run(
    f"sh -c \"{compileCommand} && ./a.out\"",
    workdir="/usr/src",
    stdin=True, stdout=True, stderr=True, socket=True
)

sock = res.output._sock
sock.sendall(code.encode())
sock.shutdown(socket.SHUT_WR)
while chunk:=sock.recv(1024):
    print(chunk.decode(errors="replace"), end="")

      �Warning, results might be unstable:
* CPU frequency scaling enabled: CPU 0 between 1,000.0 and 3,900.0 MHz
* Turbo is enabled, CPU frequency will fluctuate

Recommendations
* Use 'pyperf system tune' before benchmarking. See https://github.com/psf/pyperf
      �
|               ns/op |                op/s |    err% |     total | benchmark
|--------------------:|--------------------:|--------:|----------:|:----------
|              504.22 |        1,983,269.92 |    0.1% |      0.01 | `test`


In [ ]:
"""
      �Warning, results might be unstable:
* CPU frequency scaling enabled: CPU 0 between 1,000.0 and 3,900.0 MHz
* Turbo is enabled, CPU frequency will fluctuate

Recommendations
* Use 'pyperf system tune' before benchmarking. See https://github.com/psf/pyperf
�
|               ns/op |                op/s |    err% |     total | benchmark
|--------------------:|--------------------:|--------:|----------:|:----------
|              506.90 |        1,972,793.71 |    0.4% |      0.01 | `test`
"""


In [ ]:
sock.write()

ValueError: I/O operation on closed file.